In [1]:
import asyncio
import math
import numpy
import astropy.coordinates
import astropy.time
import astropy.units as u

from lvmtipo.site import Site
from lvmtipo.siderostat import Siderostat
from lvmtipo.fiber import Fiber
from lvmtipo.target import Target
from lvmtipo.actors import lvm

device_module = 2
derot_buffer = 600

await lvm.sci.km.start()
await lvm.sci.pwi.start()
await lvm.sci.agc.start()
await lvm.spec.agc.start()

await lvm.sci.pwi.setConnected(True)
await lvm.skye.pwi.start()
await lvm.skye.pwi.setConnected(True, statusTick=1.0)

print(await lvm.sci.km.status())
print(await lvm.sci.pwi.status())
print(await lvm.sci.agc.status())
print(await lvm.spec.agc.status())

{'Reachable': True, 'AtHome': True, 'Moving': False, 'PositionSwitchStatus': 254, 'Position': -135.0, 'DeviceEncoder': {'Position': 0.0, 'Unit': 'STEPS'}, 'Velocity': 0.0, 'AtLimit': False, 'CurrentTime': 1676765815992173.0, 'Simulate': True, 'SkyPA': -270.0}
{'is_tracking': False, 'is_connected': True, 'is_slewing': False, 'is_enabled': True, 'ra_j2000_hours': 5.49142631101966, 'dec_j2000_degs': -74.0415599874499, 'ra_apparent_hours': 5.48212896637309, 'dec_apparent_degs': -74.0275786416657, 'altitude_degs': 45.0, 'azimuth_degs': 180.0, 'field_angle_rate_at_target_degs_per_sec': 0.0, 'field_angle_here_degs': 3.09300313421224e-05, 'field_angle_at_target_degs': 0.0, 'axis0': {'dist_to_target_arcsec': 0.0, 'is_enabled': True, 'position_degs': 180.0, 'rms_error_arcsec': 0.0, 'servo_error_arcsec': 0.0}, 'axis1': {'dist_to_target_arcsec': 0.0, 'is_enabled': True, 'position_degs': 45.0, 'rms_error_arcsec': 0.0, 'servo_error_arcsec': 0.0}, 'model': {'filename': '', 'num_points_enabled': 0, 'n

In [2]:
await lvm.sci.pwi.status()

{'is_tracking': False,
 'is_connected': True,
 'is_slewing': False,
 'is_enabled': True,
 'ra_j2000_hours': 5.49145363271117,
 'dec_j2000_degs': -74.0415590994674,
 'ra_apparent_hours': 5.48215626533269,
 'dec_apparent_degs': -74.0275786416657,
 'altitude_degs': 45.0,
 'azimuth_degs': 180.0,
 'field_angle_rate_at_target_degs_per_sec': 0.0,
 'field_angle_here_degs': -1.58036228696702e-07,
 'field_angle_at_target_degs': 0.0,
 'axis0': {'dist_to_target_arcsec': 0.0,
  'is_enabled': True,
  'position_degs': 180.0,
  'rms_error_arcsec': 0.0,
  'servo_error_arcsec': 0.0},
 'axis1': {'dist_to_target_arcsec': 0.0,
  'is_enabled': True,
  'position_degs': 45.0,
  'rms_error_arcsec': 0.0,
  'servo_error_arcsec': 0.0},
 'model': {'filename': '',
  'num_points_enabled': 0,
  'num_points_total': 0,
  'rms_error_arcsec': 0.0},
 'geometry': 0}

In [3]:
from IPython.display import clear_output
from cluplus.proxy import invoke

await lvm.sci.km.moveAbsolute(0, "DEG")

exptime = 5
rc = await invoke(lvm.sci.agc.expose(exptime), lvm.spec.agc.expose(exptime+0.5))
filenames = rc.flatten().unpack("*.filename") 
print(filenames)


['/data/lvm/sci/agcam/east/20230219/lvm.sci.agcam.east_00000019.fits', '/data/lvm/sci/agcam/center/20230219/lvm.sci.agcam.center_00000019.fits', '/data/lvm/sci/agcam/west/20230219/lvm.sci.agcam.west_00000019.fits', '/data/lvm/spec/agcam/east/20230219/lvm.spec.agcam.east_00000001.fits']


In [4]:
exptime = 1
rc = await invoke(lvm.sci.agc.expose(exptime), lvm.spec.agc.expose(exptime+0.5))
filenames = rc.flatten().unpack("*.filename") 
print(filenames)

['/data/lvm/sci/agcam/east/20230219/lvm.sci.agcam.east_00000020.fits', '/data/lvm/sci/agcam/center/20230219/lvm.sci.agcam.center_00000020.fits', '/data/lvm/sci/agcam/west/20230219/lvm.sci.agcam.west_00000020.fits', '/data/lvm/spec/agcam/east/20230219/lvm.spec.agcam.east_00000002.fits']


In [ ]:

azang = 180.0
medSign = -1
sid = Siderostat(azang=azang, medSign=medSign)
homeOffset = 135.0
homeIsWest = False
geoloc = Site(name = "MPIA")

ra_h, dec_d = (await lvm.sci.pwi.status()).flatten().unpack("ra_j2000_hours", "dec_j2000_degs")

targ = astropy.coordinates.SkyCoord(ra=ra_h, dec=dec_d, unit=(u.hourangle, u.deg))
target = Target(targ)

delta_time=30
polynoms=10

now = astropy.time.Time.now()
traj = sid.mpiaMocon(geoloc, 
              target,
              None,
              deltaTime=delta_time,
              homeIsWest=homeIsWest,
              homeOffset=homeOffset,
              polyN=polynoms,
              time=now)

print(targ)
print(traj)

In [ ]:

await lvm.sci.km.moveAbsolute(traj[0][2], "STEPS")

# clear buffer
try:
   print(await lvm.sci.km.chat(1, 226, device_module))

except Exception as ex:
   pass

# create buffer
#print (await lvm.sci.km.chat(1, 220, device_module, derot_buffer))
print (await lvm.sci.km.chat(1, 220, device_module, derot_buffer, 0))

# upload trajectory
for i, t in enumerate(traj):
#   print (await lvm.sci.km.chat(1, 221, device_module, 0, f"{i} {t[0]} {t[1]} {t[2]} {t[3]} {t[4]}"))
   print (await lvm.sci.km.chat(1, 221, device_module, 0, f"{i} {t[0]} {t[1]} {t[2]}"))

# start profile
print(await lvm.sci.km.chat(1, 222, device_module, 0))


In [ ]:
from IPython.display import clear_output
from cluplus.proxy import invoke

exptime = 5
moidx = 0

while moidx < len(traj):
    try:
#        clear_output(wait=True)
        rc = (await lvm.sci.km.chat(1, 225, device_module)).unpack()
        moidx = int(rc[0].split(' ')[-1])
        print(f"{moidx} {(await lvm.sci.km.getPosition()).unpack()}")
#        await asyncio.sleep(1.0)
        rc = await invoke(lvm.sci.agc.expose(exptime), lvm.spec.agc.expose(exptime+0.5))
        filenames = rc.flatten().unpack("*.filename") 
        print(filenames)
        
    except Exception as ex:
       print(ex)
#       break


In [ ]:
print(await lvm.sci.km.chat(1, 224, device_module))

In [ ]:
from IPython.display import clear_output
from cluplus.proxy import invoke

exptime=5.0

for i in range(20):
    try:
        rc = await invoke(lvm.sci.agc.expose(exptime), lvm.spec.agc.expose(exptime+0.5))
        filenames = rc.flatten().unpack("*.filename") 
        print(filenames)
        
    except Exception as ex:
       print(ex)


In [138]:
ra_h, dec_d = (await lvm.sci.pwi.status()).flatten().unpack("ra_j2000_hours", "dec_j2000_degs")

await lvm.sci.km.slewStart(ra_h, dec_d)

{'Reachable': True,
 'AtHome': False,
 'Moving': False,
 'PositionSwitchStatus': 0,
 'Position': 83.98016664476864,
 'DeviceEncoder': {'Position': 3941643.0, 'Unit': 'STEPS'},
 'Velocity': 0.0,
 'AtLimit': False,
 'CurrentTime': 164209029.0,
 'Simulate': False,
 'SkyPA': 167.9603333289537}

In [139]:
await lvm.sci.km.slewStop()


{'Reachable': True,
 'AtHome': False,
 'Moving': False,
 'PositionSwitchStatus': 0,
 'Position': 81.76422220054579,
 'DeviceEncoder': {'Position': 3901756.0, 'Unit': 'STEPS'},
 'Velocity': 0.0,
 'AtLimit': False,
 'CurrentTime': 167678999.0,
 'Simulate': False,
 'SkyPA': 163.52844444010918}

In [132]:
await lvm.sci.km.isMoving()

{'Moving': False}

In [133]:
await lvm.sci.km.status()

{'Reachable': True,
 'AtHome': False,
 'Moving': False,
 'PositionSwitchStatus': 0,
 'Position': -53.03816667486285,
 'DeviceEncoder': {'Position': 1475313.0, 'Unit': 'STEPS'},
 'Velocity': 0.0,
 'AtLimit': False,
 'CurrentTime': 162699831.0,
 'Simulate': False,
 'SkyPA': -106.07633333497256}